In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
def evolution(state, operations, coefficient=1, verbose=False):
    """
    Args:
        state (tuple): State.
        operations (str): String consisting of 'i', 'h' and 'k'.
        coefficient (complex): Coefficient by given state.

    """
    def apply_k(value: int):
        assert value in [0, 1]

        res = dict()
        if value == 0:
            res[0] = 1 / np.sqrt(2)
            res[1] = 1 / np.sqrt(2)
        elif value == 1:
            res[0] = -1j / np.sqrt(2)
            res[1] = 1j / np.sqrt(2)
        return res


    def apply_h(value: int):
        assert value in [0, 1]

        res = dict()
        if value == 0:
            res[0] = 1 / np.sqrt(2)
            res[1] = 1 / np.sqrt(2)
        elif value == 1:
            res[0] = 1 / np.sqrt(2)
            res[1] = -1 / np.sqrt(2)
        return res

    all_states = dict()
    all_states[state] = coefficient
    
    h_indices = [x[0] for x in enumerate(operations) if x[1] == 'h']
    k_indices = [x[0] for x in enumerate(operations) if x[1] == 'k']
    for indices, apply, letter in [(h_indices, apply_h, 'H'), (k_indices, apply_k, 'K')]:
        for i in indices:
            # It's necessary to save keys before iterating over dict.
            all_states_copy = all_states.copy()
            for s in all_states_copy:
                # Apply operation to i'th qubit.
                applied = apply(s[i])
                for v in applied:
                    tmp = s[:i] + (v,) + s[i + 1:]
                    if tmp in all_states:
                        all_states[tmp] *= applied[v]
                    else:
                        all_states[tmp] = all_states_copy[s] * applied[v]
            if verbose:
                print(letter, i)
                print(all_states)

    if verbose:
        print('=======')
        print('Result:')
        print(all_states)
        
    return all_states

In [127]:
operations = 'hhi'

state1 = (0, 1, 1)
ampl1 = 0.2
phase1 = 1.2
res1 = evolution(state1, operations, ampl1 * np.exp(1j * phase1))
res1

{(0, 0, 1): (0.03623577544766736+0.09320390859672262j),
 (0, 1, 1): (-0.03623577544766736-0.09320390859672262j),
 (1, 0, 1): (0.03623577544766736+0.09320390859672262j),
 (1, 1, 1): (-0.03623577544766736-0.09320390859672262j)}

In [128]:
state2 = (1, 0, 1)
# ampl2 = -0.1
# phase2 = 0.9
ampl2 = 0.2
phase2 = 1.2
res2 = evolution(state2, operations, ampl2 * np.exp(1j * phase2))
res2

{(0, 0, 1): (0.03623577544766736+0.09320390859672262j),
 (0, 1, 1): (0.03623577544766736+0.09320390859672262j),
 (1, 0, 1): (-0.03623577544766736-0.09320390859672262j),
 (1, 1, 1): (-0.03623577544766736-0.09320390859672262j)}

In [129]:
def merge_dicts(dict1, dict2):
    tmp = {k: dict1.get(k, 0) + dict2.get(k, 0) for k in set(dict1) | set(dict2)}
    res = {k: tmp[k] for k in tmp if abs(tmp[k]) > 0.0}
    return res

In [130]:
merged = merge_dicts(res1, res2)

In [131]:
merged

{(0, 0, 1): (0.07247155089533472+0.18640781719344524j),
 (1, 1, 1): (-0.07247155089533472-0.18640781719344524j)}

In [152]:
operations = 'hhi'
quantum_system = [(0,0,0), (1,0,1), (1,1,1)]
amplitudes = [-0.1, 0.4, 0.9]
phases = [0.2, 0.3, 0.4]

total = dict()
for i in range(len(quantum_system)):
    tmp = evolution(quantum_system[i], operations, amplitudes[i] * np.exp(1j * phases[i]))
    total = merge_dicts(total, tmp)
    
total

{(0, 0, 0): (-0.049003328892062076-0.00993346653975306j),
 (0, 0, 1): (0.6055447451264193+0.2343422953711606j),
 (0, 1, 0): (-0.049003328892062076-0.00993346653975306j),
 (0, 1, 1): (-0.22341014947617704-0.1161342127066248j),
 (1, 0, 0): (-0.049003328892062076-0.00993346653975306j),
 (1, 0, 1): (-0.6055447451264193-0.2343422953711606j),
 (1, 1, 0): (-0.049003328892062076-0.00993346653975306j),
 (1, 1, 1): (0.22341014947617704+0.1161342127066248j)}

In [136]:
def random_phases(size):
    return 2 * np.pi * np.random.random(size)

In [137]:
num_units = 3

thetas = random_phases(2 ** num_units)
thetas

array([5.52340384, 2.90789221, 2.00846437, 4.57811354, 5.68869502,
       4.63920372, 1.60674179, 0.58597221])